### 1. Pytorch detach

+ 返回一个新的 ```tensor```，从当前计算图中分离下来的，但是仍指向原变量的存放位置，不同之处只是requires_grad为false，得到的这个tensor永远不需要计算其梯度，不具有 ```grad```。
+ 使用 ```detach``` 返回的```tensor```和原始的```tensor```共同一个内存，即一个修改另一个也会跟着改变。
+ 当我们再训练网络的时候可能希望保持一部分的网络参数不变，只对其中一部分的参数进行调整；或者值训练部分分支网络，并不让其梯度对主网络的梯度造成影响，这时候我们就需要使用detach()函数来切断一些分支的反向传播

In [29]:
import torch

a = torch.tensor([1, 2, 4.], requires_grad = True)
print('{0: <7}'.format('a:'), a.grad)

out = a.sigmoid()
print('{0: <7}'.format('out:'),out)
out.sum().backward()
print('{0: <7}'.format('a.grad:'),a.grad)

## add detach
c = out.detach()
print('{0: <7}'.format('c:'),c)

# c.sum().backward() # can not backward

a:      None
out:    tensor([0.7311, 0.8808, 0.9820], grad_fn=<SigmoidBackward>)
a.grad: tensor([0.1966, 0.1050, 0.0177])
c:      tensor([0.7311, 0.8808, 0.9820])


In [50]:
# cutoff backward by detach
def round_pass(x):
    y = x.round()
    y_grad = x
    return y.detach() - y_grad.detach() + y_grad

a = torch.tensor([1, 2, 4.5], requires_grad = True)
a = 2 * a
m = round_pass(a)
m.sum().backward()
print(a,'\n',a.grad)

print("*"*60)
x = torch.tensor([1, 2, 4.5], requires_grad = True)
y = torch.tensor([1, 2, 4.5], requires_grad = True)
z = round_pass(x) + y
z.sum().backward()
print('{0:<7}'.format("z:"), x)
print('{0:<7}'.format("x.grad:"), x.grad)
print('{0:<7}'.format("y.grad:"), x.grad)

tensor([2., 4., 9.], grad_fn=<MulBackward0>) 
 None
************************************************************
z:      tensor([1.0000, 2.0000, 4.5000], requires_grad=True)
x.grad: tensor([1., 1., 1.])
y.grad: tensor([1., 1., 1.])


/usr/local/bin/ipython:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
